In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pip install -U -q "google-generativeai>=0.8.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 20.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("Explain AI to me like I'm a kid")

In [6]:
Markdown(response.text)

Imagine you have a really smart puppy.  You teach it tricks, like "sit" and "fetch".  The more you teach it, the better it gets at those tricks, right?

AI is kind of like that, but instead of a puppy, it's a computer program.  We teach the computer program by showing it lots and lots of examples.  For example, we might show it thousands of pictures of cats and dogs, and tell it which is which.  Eventually, the computer program gets so good at it, it can look at a new picture and tell you if it's a cat or a dog all by itself!

That's AI – it's a computer program that can learn and get better at things without us directly telling it how to do every single step.  It can do lots of cool things, like:

* **Play games:**  Think of video game characters that seem really smart and can beat you!
* **Understand your voice:**  Like when you talk to Siri or Alexa.
* **Translate languages:**  Turning English into Spanish, or French into Chinese.
* **Recommend movies:**  Netflix or YouTube suggesting things you might like.

It's still learning and growing, just like a puppy, but it's already pretty amazing!


In [7]:
chat = flash.start_chat(history=[])
response = chat.send_message('Hello! my name is Tobi.')
print(response.text)
Markdown(response.text)

Hi Tobi! Nice to meet you.



Hi Tobi! Nice to meet you.


In [8]:
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/embedding-001
models/text-embedding-004
models

In [9]:
for model in genai.list_models():
    if model.name == 'models/gemini-1.5-flash':
        print(model)
        break

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description=('Alias that points to the most recent stable version of Gemini 1.5 Flash, our '
                   'fast and versatile multimodal model for scaling across diverse tasks.'),
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


In [10]:
short_model_prompt = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config = genai.GenerationConfig(max_output_tokens=200)

)

response = short_model_prompt.generate_content("Write a 100 word essay on importance of olives in modern society")
print(response.text)

Olives, beyond their culinary significance, hold considerable importance in modern society.  Their cultivation supports economies in Mediterranean regions, providing livelihoods for farmers and processors.  Olive oil, a staple ingredient, is celebrated for its health benefits, containing monounsaturated fats linked to cardiovascular health.  The versatile fruit itself, enjoyed pickled or marinated, adds flavor and texture to diverse cuisines worldwide.  Furthermore, olive production contributes to sustainable agriculture, often utilizing traditional methods that minimize environmental impact.  In essence, olives' economic, health, and environmental contributions solidify their vital role in contemporary life.



In [15]:
from google.api_core import retry

high_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config = genai.GenerationConfig(temperature=0.0)
)

retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}
for _ in range(5):
    response = high_temp_model.generate_content('Pick a random color .... (respond in a single word)',
                                                    request_options=retry_policy)
    if response.parts:
        print(response.text, '-' * 25)

Maroon
 -------------------------
Maroon
 -------------------------
Maroon
 -------------------------
Maroon
 -------------------------
Maroon
 -------------------------


In [14]:
low_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0))

for _ in range(5):
  response = low_temp_model.generate_content('Pick a random colour... (respond in a single word)',
                                             request_options=retry_policy)
  if response.parts:
    print(response.text, '-' * 25)

Maroon
 -------------------------
Maroon
 -------------------------
Maroon
 -------------------------
Maroon
 -------------------------
Maroon
 -------------------------


In [17]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config = genai.GenerationConfig(
        temperature = 1.0,
        top_k = 6,
        top_p = 0.95,
    )
)

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = model.generate_content(story_prompt, request_options=retry_policy)
print(response.text)

Bartholomew was not your average cat. He wasn't content with napping in sunbeams and chasing dust motes. Bartholomew yearned for adventure, for the unknown. He spent his days gazing out the window, watching birds flitting through the branches, their carefree lives fueling his wanderlust.

One day, a delivery truck arrived, bearing a package that smelled of faraway lands and exciting possibilities. As the delivery man placed the box on the porch, Bartholomew saw his chance. With a flick of his tail, he slipped out the open door and onto the porch. The man, oblivious, went on his way, leaving Bartholomew alone with the tempting box.

Sniffing cautiously, Bartholomew found a small, raggedy hole near the bottom.  He squeezed through, finding himself inside a wonderland of exotic scents and textures. He was inside a sack of coffee beans! Bartholomew burrowed deep, reveling in the rich aroma, and then, with a jolt, the sack was lifted. He was being carried!

Bartholomew's heart thumped. He h

In [18]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

Sentiment: **POSITIVE**


In [19]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ))

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

positive


In [21]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"


response = model.generate_content([few_shot_prompt, customer_order], request_options=retry_policy)
print(response.text)

```json
{
  "size": "large",
  "type": "normal",
  "ingredients": ["cheese", "pineapple"]
}
```



In [ ]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ))

response = model.generate_content("Can I have a large dessert pizza with apple and chocolate")
print(response.text)